In [2]:
import torch
from torchsummary import summary
from mcunet.model_zoo import build_model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for model_id in ("mcunet-in2", "mcunet-in4"):
    model, resolution, _ = build_model(model_id, pretrained=True)
    model = model.to(device)
    print(f"\n===== {model_id} (res：{resolution}×{resolution}) =====")

    summary(model, input_size=(3, resolution, resolution), device=str(device))

c:\Users\96156\OneDrive - nyu.edu\ce\9483\Efficient_ML_mcunet\mcunet\model_zoo.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(download_url(sd_url), map


===== mcunet-in2 (res：160×160) =====
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 80, 80]             432
       BatchNorm2d-2           [-1, 16, 80, 80]              32
             ReLU6-3           [-1, 16, 80, 80]               0
         ConvLayer-4           [-1, 16, 80, 80]               0
            Conv2d-5           [-1, 16, 80, 80]             144
       BatchNorm2d-6           [-1, 16, 80, 80]              32
             ReLU6-7           [-1, 16, 80, 80]               0
            Conv2d-8            [-1, 8, 80, 80]             128
       BatchNorm2d-9            [-1, 8, 80, 80]              16
MBInvertedConvLayer-10            [-1, 8, 80, 80]               0
MobileInvertedResidualBlock-11            [-1, 8, 80, 80]               0
           Conv2d-12           [-1, 24, 80, 80]             192
      BatchNorm2d-13           [-1, 24, 80, 80]      

In [3]:
import torch
import time
import math
import copy
import random
import logging
import requests
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.utils.data import DataLoader
from thop import profile
from thop import clever_format
from torchsummary import summary
from mcunet.model_zoo import build_model

In [ ]:
class CIFAR:
    """
    CIFAR-10 DataLoader wrapper for MCUNet (160×160 input, finetune on ImageNet-pretrained weights).
    """
    def __init__(self, batch_size=128, num_workers=2):
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_loader, self.test_loader = self.prepare_data()

    def prepare_data(self):
        # 1) 训练集 transforms
        transform_train = transforms.Compose([
            transforms.Resize((160, 160)),              
            transforms.RandomCrop(160, padding=16),     
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(              
                [0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225],
            ),
        ])

        # 2) 测试集 transforms
        transform_test = transforms.Compose([
            transforms.Resize((160, 160)),      
            transforms.ToTensor(),
            transforms.Normalize( 
                [0.485, 0.456, 0.406],
                [0.229, 0.224, 0.225],
            ),
        ])

        trainset = torchvision.datasets.CIFAR10(
            root="./data", train=True, download=True, transform=transform_train
        )
        trainloader = torch.utils.data.DataLoader(
            trainset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

        testset = torchvision.datasets.CIFAR10(
            root="./data", train=False, download=True, transform=transform_test
        )
        testloader = torch.utils.data.DataLoader(
            testset,
            batch_size=self.batch_size,   
            shuffle=False,
            num_workers=self.num_workers,
        )

        return trainloader, testloader


In [6]:
dataloader = {}
dataloader['train'], dataloader['test'] = CIFAR().train_loader, CIFAR().test_loader

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [30]:
import sys, torch
print("Executable:", sys.executable)
print("Torch location:", torch.__file__)
print("CUDA version in torch:", torch.version.cuda)
print("CUDA available?:", torch.cuda.is_available())

Executable: c:\Users\96156\anaconda3\python.exe
Torch location: c:\Users\96156\anaconda3\Lib\site-packages\torch\__init__.py
CUDA version in torch: 12.1
CUDA available?: True


In [39]:
device='cuda'
model_name="mcunet-in2"
model_save_path=f"/content/{model_name}_cifar10.pth"
model,resolution,desc=build_model(model_name)
in_features=model.classifier.in_features
model.classifier=nn.Linear(in_features,10)
model=model.to(device)
summary(model, input_size=(3, 160, 160))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 80, 80]             432
       BatchNorm2d-2           [-1, 16, 80, 80]              32
             ReLU6-3           [-1, 16, 80, 80]               0
         ConvLayer-4           [-1, 16, 80, 80]               0
            Conv2d-5           [-1, 16, 80, 80]             144
       BatchNorm2d-6           [-1, 16, 80, 80]              32
             ReLU6-7           [-1, 16, 80, 80]               0
            Conv2d-8            [-1, 8, 80, 80]             128
       BatchNorm2d-9            [-1, 8, 80, 80]              16
MBInvertedConvLayer-10            [-1, 8, 80, 80]               0
MobileInvertedResidualBlock-11            [-1, 8, 80, 80]               0
           Conv2d-12           [-1, 24, 80, 80]             192
      BatchNorm2d-13           [-1, 24, 80, 80]              48
            ReLU6-14       

In [8]:
def train(model: nn.Module, train_dataloader: DataLoader, lr=0.1, epochs=10, device="cuda"):
    """
    Training function for training the model using the given dataloader, optimizer, and loss function.

    Args:
        model: The neural network model to be trained.
        train_dataloader: The DataLoader for the training data.
        lr: The learning rate for the optimizer.
        epochs: Number of epochs to train the model.
        device: Device to run the training on ('cuda' or 'cpu').

    Returns:
        None. Prints training progress for each epoch.
    """
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(
        model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs
    )

    for epoch in range(epochs):
        print(f"\nEpoch: {epoch+1}/{epochs}")
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        with tqdm(total=len(train_dataloader), desc=f"Train Epoch {epoch+1}") as pbar:
            for batch_idx, (inputs, targets) in enumerate(train_dataloader):
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = model(inputs)

                # Compute the loss
                loss = criterion(outputs, targets)

                # Backward pass (backpropagation)
                loss.backward()

                # Optimizer step (update model parameters)
                optimizer.step()

                # Update the running loss and accuracy
                train_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()

                pbar.set_postfix(
                    {
                        "Loss": f"{train_loss/(batch_idx+1):.3f}",
                        "Acc": f"{100.0 * correct / total:.3f}%",
                    }
                )
                pbar.update(1)
        scheduler.step()

        epoch_loss = train_loss / len(train_dataloader)
        epoch_acc = 100.0 * correct / total
        print(
            f"Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%"
        )

In [40]:
device = "cuda" 
train(model, dataloader['train'], lr=0.01, epochs=6, device=device)


Epoch: 1/6


Train Epoch 1: 100%|██████████| 391/391 [01:03<00:00,  6.13it/s, Loss=0.456, Acc=84.026%]


Epoch [1/6] - Loss: 0.4561, Accuracy: 84.03%

Epoch: 2/6


Train Epoch 2: 100%|██████████| 391/391 [01:04<00:00,  6.06it/s, Loss=0.239, Acc=91.690%]


Epoch [2/6] - Loss: 0.2391, Accuracy: 91.69%

Epoch: 3/6


...

Epoch [4/6] - Loss: 0.1324, Accuracy: 95.35%

Epoch: 5/6


Train Epoch 5: 100%|██████████| 391/391 [01:04<00:00,  6.08it/s, Loss=0.097, Acc=96.716%]


Epoch [5/6] - Loss: 0.0966, Accuracy: 96.72%

Epoch: 6/6


Train Epoch 6: 100%|██████████| 391/391 [01:04<00:00,  6.08it/s, Loss=0.081, Acc=97.254%]

Epoch [6/6] - Loss: 0.0810, Accuracy: 97.25%


In [9]:
def evaluate(model: nn.Module, test_dataloader: DataLoader, device=device) -> float:
    """
    Inference function to evaluate the model on a test dataset using the provided dataloader,
    with tqdm progress bar for visualization.

    Args:
        model: The neural network model to be used for inference.
        test_dataloader: The dataloader for the test data.
        device: Device to use ('cuda' or 'cpu').

    Returns:
        None: Prints the accuracy on the test set.
    """
    model = model.to(device)
    model.eval()

    correct = 0
    total = 0

    # Disable gradient computation during inference
    with torch.no_grad():
        for inputs, targets in tqdm(test_dataloader, desc="Evaluating", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            _, predicted = outputs.max(1)

            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    accuracy = 100.0 * correct / total
    # print(f"Accuracy on the test set: {accuracy:.2f}%")
    return accuracy

In [41]:
accuracy = evaluate(model, dataloader['test'])
print(f'\n Accuracy: {accuracy}%')


 Accuracy: 94.1%


In [42]:
import os

model_save_path = f"C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/{model_name}_2_cifar10.pth"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
torch.save(model.state_dict(), model_save_path)
print("Saved to:", model_save_path)

Saved to: C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/mcunet-in2_2_cifar10.pth


In [ ]:
model_name="mcunet-in4"
model_save_path = f"C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/{model_name}_2_cifar10.pth"
model,resolution,desc=build_model(model_name)
in_features=model.classifier.in_features
model.classifier=nn.Linear(in_features,10)

model=model.to(device)
summary(model, input_size=(3, 160, 160))



Epoch: 1/6


Train Epoch 1: 100%|██████████| 391/391 [01:23<00:00,  4.69it/s, Loss=0.401, Acc=86.080%]


Epoch [1/6] - Loss: 0.4009, Accuracy: 86.08%

Epoch: 2/6


Train Epoch 2: 100%|██████████| 391/391 [01:25<00:00,  4.57it/s, Loss=0.172, Acc=94.074%]


Epoch [2/6] - Loss: 0.1720, Accuracy: 94.07%

Epoch: 3/6


...

Epoch [4/6] - Loss: 0.0745, Accuracy: 97.41%

Epoch: 5/6


Train Epoch 5: 100%|██████████| 391/391 [01:21<00:00,  4.77it/s, Loss=0.053, Acc=98.250%]


Epoch [5/6] - Loss: 0.0532, Accuracy: 98.25%

Epoch: 6/6


Train Epoch 6: 100%|██████████| 391/391 [01:21<00:00,  4.78it/s, Loss=0.042, Acc=98.730%]

Epoch [6/6] - Loss: 0.0420, Accuracy: 98.73%


In [ ]:
device = "cuda" 
train(model, dataloader['train'], lr=0.01, epochs=6, device=device)

In [37]:
accuracy = evaluate(model, dataloader['test'])
print(f'\n Accuracy: {accuracy}%')


 Accuracy: 95.69%


In [38]:
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
torch.save(model.state_dict(), model_save_path)
print("Saved to:", model_save_path)

Saved to: C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/mcunet-in4_2_cifar10.pth


In [23]:
MODEL_NAME   = 'mcunet-in4'
MODEL_PATH   = 'content/mcunet-in4_2_cifar10.pth'
def load_pretrained_model(model_name: str, weight_path: str, num_classes=10, device=device):
    """
    Load a pre-trained MCUNet model and prepare it for inference.

    Args:
        model_name (str): e.g., "mcunet-in4"
        weight_path (str): Path to the saved .pth file
        num_classes (int): Number of output classes (CIFAR-10 -> 10)
        device (str): 'cuda' or 'cpu'

    Returns:
        nn.Module: Loaded model
    """

    model, resolution, desc = build_model(model_name)


    if hasattr(model, 'classifier'):
        in_features = model.classifier.in_features
        model.classifier = nn.Linear(in_features, num_classes)
    elif hasattr(model, 'fc'):
        in_features = model.fc.in_features
        model.fc = nn.Linear(in_features, num_classes)
    else:
        raise ValueError("Cannot find final classification layer in model")


    state_dict = torch.load(weight_path, map_location=device)
    model.load_state_dict(state_dict)

    model.to(device)
    model.eval()

    return model

model = load_pretrained_model(MODEL_NAME, MODEL_PATH)
print("✅ Model load successful")

✅ Model load successful


C:\Users\96156\AppData\Local\Temp\ipykernel_72552\453617923.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


In [22]:
accuracy = evaluate(model, dataloader['test'])
print(f'\n Accuracy: {accuracy}%')


 Accuracy: 10.98%


In [10]:
def evaluate_with_inftime(model: nn.Module, test_dataloader: DataLoader, device="cuda") -> tuple[float, float]:
    """
    Inference function to evaluate the model on a test dataset using the provided dataloader,
    with tqdm progress bar for visualization. It also measures average inference time per batch.

    Args:
        model: The neural network model to be used for inference.
        test_dataloader: The dataloader for the test data.
        device: Device to use ('cuda' or 'cpu').

    Returns:
        Tuple of:
        - accuracy (float): Top-1 accuracy on the test set (%)
        - avg_inference_time (float): Average inference time per batch (ms)
    """
    model = model.to(device)
    model.eval()

    correct = 0
    total = 0
    inference_times = []

    with torch.no_grad():
        for inputs, targets in tqdm(test_dataloader, desc="Evaluating", leave=False):
            inputs, targets = inputs.to(device), targets.to(device)

            # Start timing
            start_time = time.perf_counter()
            outputs = model(inputs)
            if device == "cuda":
                torch.cuda.synchronize()  # 等待GPU完成任务
            end_time = time.perf_counter()

            inference_times.append((end_time - start_time) * 1000)  # 转为毫秒

            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    accuracy = 100.0 * correct / total
    avg_infer_time = sum(inference_times) / total

    return accuracy, avg_infer_time

In [17]:
acc, avg_time = evaluate_with_inftime(model, dataloader['test'],device="cuda")
print(f"✅ Accuracy: {acc:.2f}%")
print(f"⏱️ Avg inference time per batch: {avg_time:.2f} ms")

✅ Accuracy: 95.69%
⏱️ Avg inference time per batch: 0.42 ms


In [104]:
def calculate_sparsity(model: nn.Module) -> float:
    """
    Calculate the sparsity of a given model (proportion of zero weights).

    Args:
        model (nn.Module): The PyTorch model to calculate sparsity for.
    Returns:
        float: The sparsity of the model (proportion of zero weights).
    """
    total_params = 0
    zero_params = 0

    # Iterate over each parameter tensor in the model
    for param in model.parameters():
        total_params += param.numel()  # Total number of parameters
        zero_params += (param == 0).sum().item()  # Count of zero weights

    sparsity = zero_params / total_params
    return sparsity

def get_sparsity(tensor: torch.Tensor) -> float:
    """
    Calculate the sparsity of a given tensor (proportion of zero weights).

    Args:
        model (torch.Tensor): The tensor to calculate sparsity for.

    Returns:
    """
    return 1 - float(tensor.count_nonzero()) / tensor.numel()

def calculate_total_parameters(model: nn.Module) -> int:
    """
    Calculate the total number of parameters in a model.

    Args:
        model (nn.Module): The PyTorch model to calculate parameters for.

    Returns:
        int: The total number of parameters in the model.
    """
    total_params = sum(p.numel() for p in model.parameters())
    return total_params

def calculate_by_thop(model: nn.Module, input_size: tuple = (1, 3, 32, 32)) -> float:
    """
    Calculate the FLOPs (Floating Point Operations) of a model.

    Args:
        model (nn.Module): The PyTorch model to calculate FLOPs for.
        input_size (tuple): The size of the input tensor, e.g., (1, 3, 32, 32) for CIFAR-10.

    Returns:
        float: The number of FLOPs for the model.
    """
    # Create a dummy input with the correct input size
    dummy_input = torch.randn(*input_size).to(next(model.parameters()).device)

    # Use thop to profile the model and calculate FLOPs
    flops, params = profile(model, inputs=(dummy_input, ))

    flops, params = clever_format([flops, params], "%.3f")

    return flops, params

In [25]:
accuracy = evaluate(model, dataloader['test'])
sparsity = calculate_sparsity(model)
flops, params = calculate_by_thop(model)
print(f'Accuracy: {accuracy}%, FLOPs: {flops}, Params: {params}, Sparsity: {sparsity}%')

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU6'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Accuracy: 95.69%, FLOPs: 5.474M, Params: 1.415M, Sparsity: 0.0%


In [26]:
def magnitude_pruning(tensor: torch.Tensor, sparsity: float) -> torch.Tensor:
    """
    Apply magnitude pruning to a given tensor.

    Args:
        tensor (torch.Tensor): The input tensor to be pruned.
        sparsity (float): The desired spars
    Returns:
        pruned_tensor (torch.Tensor): The pruned tensor.
        mask (torch.Tensor): The pruning mask (where True means the weight is kept, and False means it is pruned).
    """
    sparsity = min(max(0.0, sparsity), 1.0)
    pruned_tensor = tensor.clone()
    mask = torch.zeros_like(tensor, dtype=torch.bool)

    # Step 1: Flatten the tensor
    flattened_tensor=pruned_tensor.view(-1)

    # Step 2: Calaulate pruning threshold
    threshold=torch.quantile(flattened_tensor.abs(),sparsity)

    # Step 3: Create a mask
    mask=(tensor.abs()>=threshold)

    # Step 4: Create the pruned tensor
    pruned_tensor=tensor * mask

    # Step 5: Ensure the pruned tensor has correct shape
    pruned_tensor=pruned_tensor.view(tensor.shape)
    mask=mask.view(tensor.shape)

    mask = mask.to(tensor.device)
    pruned_tensor = pruned_tensor.to(tensor.device)

    return pruned_tensor, mask

In [27]:
class MagnitudePruner:
    def __init__(self, model: nn.Module, sparsity: float):
        self.model = model
        self.sparsity = sparsity

    def _prune_layer(self, layer_name: str, weight_tensor: torch.Tensor):
        """
        Apply magnitude pruning to a specific layer.

        Args:
            layer_name (str): Name of the layer being pruned.
            weight_tensor (torch.Tensor): The weight tensor to prune.
        """
        with torch.no_grad():
            pruned_tensor, mask = magnitude_pruning(weight_tensor, self.sparsity)
            return pruned_tensor, mask


    def prune_selected_layers(self, layer_names: list):
        """
        Apply magnitude pruning to specific layers of the model.

        Args:
            layer_names (list): A list of parameter names to prune
        """
        masks = {}
        for name, param in self.model.named_parameters():
          if param.dim() > 1 and 'layer' in name:
            if layer_names is None or name in layer_names:
                print(f"Pruning {name}")
                pruned_tensor, mask = self._prune_layer(name, param.data)
                masks[name] = mask
        return masks

In [28]:
@torch.no_grad()
def sensitivity_scan_per_layer(model: nn.Module, dataloader: torch.utils.data.DataLoader,
                               start: float, step: float, end: float,
                               device: str = device, verbose=True):
    """
    Perform a sensitivity scan by applying magnitude pruning to each individual layer of the model
    and evaluate the accuracy at different levels of sparsity for each layer.

    Args:
        model (nn.Module): The neural network model to be pruned and evaluated.
        dataloader (DataLoader): DataLoader for the test dataset.
        start (float): Starting sparsity level.
        step (float): Step size for increasing sparsity.
        end (float): Ending sparsity level.
        device (str): Device to use ('cuda' or 'cpu').
        verbose (bool): Whether to print progress and results.

    Returns:
        dict: Dictionary containing sparsity levels and corresponding accuracies for each layer.
    """
    sparsities = np.arange(start, end, step)

    layer_accuracies = {}

    model.to(device)
    target_keywords = [
    'mobile_inverted_conv.point_linear.conv',
    'mobile_inverted_conv.depth_conv.conv',
    'mobile_inverted_conv.inverted_bottleneck.conv'
]

    for name, param in model.named_parameters():
        if param.dim() > 1 and any(k in name for k in target_keywords):
            print(f"\nScanning layer: {name}")
            param_clone = param.detach().clone()

            accuracy_per_layer = []

            for sparsity in sparsities:
              ##############################################################################
              #TODO: Compute pruned_tensor and corresponding mask           #
              ##############################################################################
              # Step 1: Initialize a Pruner
                pruner=MagnitudePruner(model,sparsity)

              # Step 2: Apply exact mask on the weight tensor
                pruned_tensor,pruned_mask=pruner._prune_layer(name,param.data)
                param.data.copy_(pruned_tensor)

              # Step 3: Evaluate model accuracy on test_dataloader
                accuracy=evaluate(model,dataloader,device)
              ##############################################################################
              #END OF YOUR CODE                              #
              ##############################################################################
                accuracy_per_layer.append(accuracy)

                if verbose:
                    print(f"Sparsity: {sparsity:.2f} | Accuracy: {accuracy:.2f}%")

                param.data.copy_(param_clone)

            layer_accuracies[name] = accuracy_per_layer
    return sparsities, layer_accuracies

In [29]:
sparsities, layer_accuracies = sensitivity_scan_per_layer(model, dataloader['test'], 0.4, 0.1, 1.0)


Scanning layer: blocks.0.mobile_inverted_conv.depth_conv.conv.weight


Sparsity: 0.40 | Accuracy: 93.80%


Sparsity: 0.50 | Accuracy: 87.36%


Sparsity: 0.60 | Accuracy: 46.42%


Sparsity: 0.70 | Accuracy: 11.90%


...

Sparsity: 0.50 | Accuracy: 95.23%


Sparsity: 0.60 | Accuracy: 94.21%


Sparsity: 0.70 | Accuracy: 94.05%


Sparsity: 0.80 | Accuracy: 93.71%


Sparsity: 0.90 | Accuracy: 82.07%


In [109]:
layer_dict = {
    # Block 0
    'blocks.0.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.0.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.0.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 1
    'blocks.1.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.1.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.1.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 2
    'blocks.2.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.2.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.2.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 3
    'blocks.3.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.3.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.3.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 4
    'blocks.4.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.4.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.4.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 5
    'blocks.5.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.5.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.5.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 6
    'blocks.6.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.6.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.6.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 7
    'blocks.7.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.7.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.7.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 8
    'blocks.8.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.8.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.8.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 9
    'blocks.9.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.9.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.9.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 10
    'blocks.10.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0,
    'blocks.10.mobile_inverted_conv.depth_conv.conv.weight': 0,
    'blocks.10.mobile_inverted_conv.point_linear.conv.weight': 0,

    # Block 11
    'blocks.11.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0.4,
    'blocks.11.mobile_inverted_conv.depth_conv.conv.weight': 0.4,
    'blocks.11.mobile_inverted_conv.point_linear.conv.weight': 0.4,

    # Block 12
    'blocks.12.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0.4,
    'blocks.12.mobile_inverted_conv.depth_conv.conv.weight': 0.4,
    'blocks.12.mobile_inverted_conv.point_linear.conv.weight': 0.4,

    # Block 13
    'blocks.13.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0.4,
    'blocks.13.mobile_inverted_conv.depth_conv.conv.weight': 0.4,
    'blocks.13.mobile_inverted_conv.point_linear.conv.weight': 0.4,

    # Block 14
    'blocks.14.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0.4,
    'blocks.14.mobile_inverted_conv.depth_conv.conv.weight': 0.4,
    'blocks.14.mobile_inverted_conv.point_linear.conv.weight': 0.4,

    # Block 15
    'blocks.15.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0.4,
    'blocks.15.mobile_inverted_conv.depth_conv.conv.weight': 0.4,
    'blocks.15.mobile_inverted_conv.point_linear.conv.weight': 0.4,

    # Block 16
    'blocks.16.mobile_inverted_conv.inverted_bottleneck.conv.weight': 0.4,
    'blocks.16.mobile_inverted_conv.depth_conv.conv.weight': 0.4,
    'blocks.16.mobile_inverted_conv.point_linear.conv.weight': 0.4,
}


In [30]:
def prune_layer_with_sparsity(weight_tensor: torch.Tensor, sparsity: float):
    """
    Apply magnitude pruning to a specific layer with a specified sparsity.

    Args:
        weight_tensor (torch.Tensor): The weight tensor to prune.
        sparsity: (float): The desired sparsity level.
    """
    with torch.no_grad():

        # 1. Flattened tensor as 1D array
        flattened_tensor=weight_tensor.view(-1)
        # 2. Calculate threshold
        threshold=torch.quantile(flattened_tensor.abs(),sparsity)
        # 3. Calculate mask
        mask=(weight_tensor.abs()>=threshold)
        # 4. Get pruned tensor
        pruned_tensor=mask * weight_tensor
        # 5. reshape pruned tensor and mask
        pruned_tensor=pruned_tensor.view(weight_tensor.shape)
        mask=mask.view(weight_tensor.shape)


        return pruned_tensor, mask

In [110]:
def recover_model():
    """
    Reload the original pretrained model from disk.
    """
    global model
    model = load_pretrained_model('mcunet-in4', 'content/mcunet-in4_2_cifar10.pth')
    print("🔁 Model recovered to original (pre-pruning) state.")


recover_model()
print(f'Sparsity before pruning: {calculate_sparsity(model)}')
for layer_name, sparsity in layer_dict.items():
    if layer_name not in dict(model.named_parameters()):
        print(f"❌ [Not Found] {layer_name}")
        continue
    print(f"✅ [Pruning] {layer_name} with sparsity {sparsity}")

    param = dict(model.named_parameters())[layer_name]
    pruned_tensor, mask = prune_layer_with_sparsity(param.data, sparsity)
    param.data.copy_(pruned_tensor)
acc = evaluate(model, dataloader['test'])
print()
print(f'after pruning: {calculate_sparsity(model)}, acuracy: {acc}%')

c:\Users\96156\OneDrive - nyu.edu\ce\9483\Efficient_ML_mcunet\mcunet\model_zoo.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(download_url(sd_url), map

🔁 Model recovered to original (pre-pruning) state.
Sparsity before pruning: 0.0
❌ [Not Found] blocks.0.mobile_inverted_conv.inverted_bottleneck.conv.weight
✅ [Pruning] blocks.0.mobile_inverted_conv.depth_conv.conv.weight with sparsity 0
✅ [Pruning] blocks.0.mobile_inverted_conv.point_linear.conv.weight with sparsity 0
✅ [Pruning] blocks.1.mobile_inverted_conv.inverted_bottleneck.conv.weight with sparsity 0
✅ [Pruning] blocks.1.mobile_inverted_conv.depth_conv.conv.weight with sparsity 0
✅ [Pruning] blocks.1.mobile_inverted_conv.point_linear.conv.weight with sparsity 0
✅ [Pruning] blocks.2.mobile_inverted_conv.inverted_bottleneck.conv.weight with sparsity 0
✅ [Pruning] blocks.2.mobile_inverted_conv.depth_conv.conv.weight with sparsity 0
✅ [Pruning] blocks.2.mobile_inverted_conv.point_linear.conv.weight with sparsity 0
✅ [Pruning] blocks.3.mobile_inverted_conv.inverted_bottleneck.conv.weight with sparsity 0
✅ [Pruning] blocks.3.mobile_inverted_conv.depth_conv.conv.weight with sparsity 0
✅


after pruning: 0.3256338127716719, acuracy: 92.66%


In [108]:
import os
model_save_path = f"C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/pruned.pth"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
torch.save(model.state_dict(), model_save_path)
print("Saved to:", model_save_path)

Saved to: C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/pruned.pth


In [58]:
recover_model()
evaluate(model, dataloader['test'])
model = model.to(device)

C:\Users\96156\AppData\Local\Temp\ipykernel_72552\453617923.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


🔁 Model recovered to original (pre-pruning) state.


In [65]:
import torch
import torch.nn as nn
import torch_pruning as tp
import copy

class MCUNetHardPruner:
    def __init__(self, model: nn.Module, layer_dict: dict, example_input: torch.Tensor):
        """
        Hard pruner for MCUNet using torch_pruning, with per-layer control.

        Args:
            model (nn.Module): The model to be pruned.
            layer_dict (dict): Dictionary of {layer_name: prune_ratio}.
            example_input (torch.Tensor): A sample input for dependency graph analysis.
        """
        self.model = copy.deepcopy(model)
        self.layer_dict = layer_dict
        self.example_input = example_input
        self.masks = {}

        self.DG = tp.DependencyGraph()
        self.DG.build_dependency(self.model, example_inputs=self.example_input)

    def prune_conv_layer(self, name: str, conv_layer: nn.Conv2d, amount: float):
        """
        Perform structured channel pruning on a single Conv2D layer.

        Args:
            name (str): Name of the layer.
            conv_layer (nn.Conv2d): The Conv2D layer to prune.
            amount (float): Ratio of channels to prune.
        """
        weight = conv_layer.weight.data
        out_channels = weight.shape[0]
        # compute L1 norm of each filter
        l1_norm = weight.abs().view(out_channels, -1).sum(dim=1)
        # number of channels to prune
        k = int(amount * out_channels)
        # pick the k smallest
        pruning_idxs = torch.argsort(l1_norm)[:k].tolist()

        group = self.DG.get_pruning_group(
            conv_layer,
            tp.prune_conv_out_channels,  # 注意：这里是 prune_conv_out_channels
            idxs=pruning_idxs
        )
        if self.DG.check_pruning_group(group):  # 可选：避免把整个通道都删光
            group.prune()
        self.masks[name] = True

    def prune_model(self):
        """
        Apply hard pruning only to specified layers in layer_dict.

        Returns:
            pruned_model (nn.Module): The structurally pruned model.
        """
        for name, module in self.model.named_modules():
            if not isinstance(module, nn.Conv2d):
                continue

            if name in self.layer_dict:
                amount = self.layer_dict[name]
                if module.out_channels >= 4 and amount > 0.0:
                    self.prune_conv_layer(name, module, amount)

        return self.model, self.masks

In [92]:
def calculate_sparsity_hp(original_model: nn.Module, pruned_model: nn.Module) -> float:
    """
    Compute the global sparsity introduced by hard pruning,
    i.e. the fraction of weights (parameters) removed.

    Args:
        original_model (nn.Module): the unpruned, pretrained model
        pruned_model   (nn.Module): the model after hard pruning

    Returns:
        sparsity (float): (orig_params - pruned_params) / orig_params
    """
    orig_params = sum(p.numel() for p in original_model.parameters())
    pruned_params = sum(p.numel() for p in pruned_model.parameters())
    return (orig_params - pruned_params) / orig_params

In [90]:
layer_dict = {
    # Blocks 0–10: 完全保留，不剪枝
    **{f'blocks.{i}.mobile_inverted_conv.inverted_bottleneck.conv': 0.0 for i in range(13)},
    **{f'blocks.{i}.mobile_inverted_conv.depth_conv.conv':          0.0 for i in range(13)},
    **{f'blocks.{i}.mobile_inverted_conv.point_linear.conv':        0.0 for i in range(13)},


    # Block 13
    'blocks.13.mobile_inverted_conv.inverted_bottleneck.conv': 0.1,
    'blocks.13.mobile_inverted_conv.depth_conv.conv':          0.1,
    'blocks.13.mobile_inverted_conv.point_linear.conv':        0.1,

    # Block 14
    'blocks.14.mobile_inverted_conv.inverted_bottleneck.conv': 0.1,
    'blocks.14.mobile_inverted_conv.depth_conv.conv':          0.10,
    'blocks.14.mobile_inverted_conv.point_linear.conv':        0.1,

    # Block 15
    'blocks.15.mobile_inverted_conv.inverted_bottleneck.conv': 0.1,
    'blocks.15.mobile_inverted_conv.depth_conv.conv':          0.1,
    'blocks.15.mobile_inverted_conv.point_linear.conv':        0.1,

    # Block 16
    'blocks.16.mobile_inverted_conv.inverted_bottleneck.conv': 0.1,
    'blocks.16.mobile_inverted_conv.depth_conv.conv':          0.1,
    'blocks.16.mobile_inverted_conv.point_linear.conv':        0.1,
}



recover_model()
example_input = torch.randn(1, 3, 160, 160, device=device)
pruner = MCUNetHardPruner(model, layer_dict, example_input)
pruned_model, masks = pruner.prune_model()

C:\Users\96156\AppData\Local\Temp\ipykernel_72552\453617923.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weight_path, map_location=device)


🔁 Model recovered to original (pre-pruning) state.


In [98]:
def iterative_prune_and_finetune(
    model: nn.Module,
    layer_dict: dict[str, float],
    example_input: torch.Tensor,
    dataloaders: dict[str, DataLoader],
    device: str = 'cuda',
    prune_steps: list[float] = [0.05, 0.10, 0.15, 0.20],
    finetune_epochs_per_step: int = 5,
    lr: float = 1e-3
) -> nn.Module:

    model = copy.deepcopy(model).to(device)

    for step_ratio, epochs in zip(prune_steps, finetune_epochs_per_step):
        # 本轮实际剪枝比例，不能超过 layer_dict 中的最大值
        this_round = {
            name: min(step_ratio, layer_dict[name])
            for name in layer_dict
        }

        # 硬剪枝
        pruner = MCUNetHardPruner(model, this_round, example_input)
        model, _ = pruner.prune_model()
        model.to(device)

        # 短期微调
        optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
        for epoch in range(epochs):
            model.train()
            for xb, yb in dataloaders['train']:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                loss = F.cross_entropy(model(xb), yb)
                loss.backward()
                optimizer.step()

            # 验证
            model.eval()
            total, correct = 0, 0
            with torch.no_grad():
                for xb, yb in dataloaders['test']:
                    xb, yb = xb.to(device), yb.to(device)
                    pred = model(xb).argmax(dim=1)
                    correct += (pred == yb).sum().item()
                    total += yb.size(0)
            acc = correct / total * 100
            print(f"[Step {step_ratio:.2f}] Epoch {epoch+1}/{epochs} — Acc: {acc:.2f}%")

    return model


In [99]:
recover_model()

layer_dict = {
    # Blocks 0–11: 不剪枝
    **{f'blocks.{i}.mobile_inverted_conv.inverted_bottleneck.conv': 0.0 for i in range(12)},
    **{f'blocks.{i}.mobile_inverted_conv.depth_conv.conv':          0.0 for i in range(12)},
    **{f'blocks.{i}.mobile_inverted_conv.point_linear.conv':        0.0 for i in range(12)},

    # Blocks 12–16: 最多 20%
    **{f'blocks.{i}.mobile_inverted_conv.inverted_bottleneck.conv': 0.2 for i in range(12, 17)},
    **{f'blocks.{i}.mobile_inverted_conv.depth_conv.conv':          0.2 for i in range(12, 17)},
    **{f'blocks.{i}.mobile_inverted_conv.point_linear.conv':        0.2 for i in range(12, 17)},
}

finetuned_model = iterative_prune_and_finetune(
    model=load_pretrained_model('mcunet-in4', 'content/mcunet-in4_2_cifar10.pth'),
    layer_dict=layer_dict,
    example_input=torch.randn(1, 3, 160, 160).to(device),
    dataloaders={'train': dataloader['train'], 'test': dataloader['test']},
    device=device,
    prune_steps=[0.05, 0.10, 0.15],
    finetune_epochs_per_step=[2, 2, 3],
    lr=1e-3
)

recover_model()
print("Final Acc:", evaluate(finetuned_model, dataloader['test'], device=device))
print("Final Sparsity:", calculate_sparsity_hp(model, finetuned_model))

c:\Users\96156\OneDrive - nyu.edu\ce\9483\Efficient_ML_mcunet\mcunet\model_zoo.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(download_url(sd_url), map

🔁 Model recovered to original (pre-pruning) state.
[Step 0.05] Epoch 1/2 — Acc: 94.53%
[Step 0.05] Epoch 2/2 — Acc: 94.79%
[Step 0.10] Epoch 1/2 — Acc: 92.59%
[Step 0.10] Epoch 2/2 — Acc: 93.28%
[Step 0.15] Epoch 1/3 — Acc: 90.43%
[Step 0.15] Epoch 2/3 — Acc: 91.57%
[Step 0.15] Epoch 3/3 — Acc: 92.00%
🔁 Model recovered to original (pre-pruning) state.


Final Acc: 92.0
Final Sparsity: 0.5660077039968902


In [102]:
import os
model_save_path = f"C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/hardpruned.pth"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
torch.save(finetuned_model.state_dict(), model_save_path)
print("Saved to:", model_save_path)

Saved to: C:/Users/96156/OneDrive - nyu.edu/ce/9483/Efficient_ML_mcunet/content/hardpruned.pth


In [111]:
torch.save(finetuned_model, "hardpruned_.pth")

In [ ]:
print("sparsity:", calculate_sparsity_hp(model,pruned_model))
print("acc:", evaluate(pruned_model, dataloader['test']))

sparsity: 0.2534402940241015


acc: 80.73
